In [23]:
# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
%env CUDA_VISIBLE_DEVICES = 1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp      # will come in handy due to the size of the data
import os.path
import random
import io
from datetime import datetime
import gc # garbage collector
import sklearn
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import math
from collections import defaultdict
import re

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

env: CUDA_VISIBLE_DEVICES=1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/WSDM/'
TRAIN_FILE = DATASET_PATH + 'train.csv'
TEST_FILE = DATASET_PATH + 'test.csv'
MEMBER_FILE = DATASET_PATH + 'members.csv'
SONG_FILE = DATASET_PATH + 'songs.csv'
SONG_EXTRA_FILE = DATASET_PATH + 'song_extra_info.csv'

In [25]:
train_data = pd.read_csv(TRAIN_FILE)
test_data = pd.read_csv(TEST_FILE)

In [4]:
member_data = pd.read_csv(MEMBER_FILE)
song_data = pd.read_csv(SONG_FILE)
song_extra_data = pd.read_csv(SONG_EXTRA_FILE)

In [5]:
def preprocess_csv_file(input_data, col_index):
    all_df = pd.DataFrame(data = None, columns=input_data.columns)
    num_col = len(input_data.columns)
    row_to_drop = list()
    for cur_row in input_data.itertuples():
        if isinstance(cur_row[col_index], str): 
            if '\n' in cur_row[col_index]:
                cur_prefix = list(cur_row)[1:col_index]          
                next_suffix = list(cur_row)[col_index + 1:]
                multi_line_list = cur_row[col_index].split('\n')
                # this is the current one, merged mannually
                all_df = all_df.append(pd.DataFrame([cur_prefix + multi_line_list[0].split(',')], columns = input_data.columns).replace(r'', np.nan), ignore_index=True)
                # this is the last one, merged mannually
                all_df = all_df.append(pd.DataFrame([multi_line_list[-1].split(',') + next_suffix], columns = input_data.columns).replace(r'', np.nan), ignore_index=True)
                # these are central ones
                for cur_record in multi_line_list[1:-1]:
                    #pass
                    all_df = all_df.append(pd.DataFrame([cur_record.split(',')], columns = input_data.columns).replace(r'', np.nan), ignore_index=True)             
                row_to_drop.append(cur_row[0])
    print('{} fixed.'.format(len(all_df.index)))
    input_data.drop(input_data.index[row_to_drop], inplace=True)
    #print(all_df)
    return input_data.append(all_df, ignore_index=True)      

In [6]:
# artist_name 4, composer 5, lyricist 6
song_data = preprocess_csv_file(song_data, 4)     
song_data = preprocess_csv_file(song_data, 5) 
song_data = preprocess_csv_file(song_data, 6) 
song_data.to_csv(DATASET_PATH + 'fix_songs.csv', index=False, sep=',')

435 fixed.
82 fixed.
0 fixed.


In [7]:
# for col_index in songs.csv: artist_name 4, composer 5, lyricist 6
# for other file please print the first tuple to check the index
def split_and_strip_append(input_data, column, col_index, save_file_name):
    all_df = pd.DataFrame(data = None, columns=[column])
    step = 0
    #for cur_row in input_data.iterrows():
    for cur_row in input_data.itertuples():
        if isinstance(cur_row[col_index], str): 
            df = pd.DataFrame([s.strip(' \"\t\s\n') for s in re.split(r'[|/]+',cur_row[col_index])], columns=[column])
            all_df = all_df.append(df, ignore_index=True)
            step += 1
            #break
            if step > 10000:
                if not os.path.isfile(save_file_name):
                    all_df.to_csv(save_file_name, mode='a', index=False, sep=' ')
                else:
                    all_df.to_csv(save_file_name, mode='a', index=False, sep=' ', header=False)
                all_df = pd.DataFrame(data = None, columns=[column])
    all_df.to_csv(save_file_name, mode='a', index=False, sep=' ', header=False)
    return all_df

In [8]:
def split_and_strip(input_data, column, col_index):
    all_df = pd.DataFrame(data = None, columns=[column])
    for cur_row in input_data.itertuples():
        if isinstance(cur_row[col_index], str): 
            df = pd.DataFrame([s.strip(' \"\t\s\n') for s in re.split(r'[|/]+',cur_row[col_index])], columns=[column])
            all_df = all_df.append(df, ignore_index=True)
            #break
    return all_df

In [9]:
# print(member_data.set_index('msno').reindex(['XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=','XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=','XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=']))

In [10]:
print(song_extra_data.head())

                                        song_id             name          isrc
0  LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=               我們  TWUM71200043
1  ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=  Let Me Love You  QMZSY1600015
2  u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=              原諒我  TWA530887303
3  92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=          Classic  USSM11301446
4  0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=             愛投羅網  TWA471306001


In [11]:
member_reorder_train = member_data.set_index('msno').reindex(train_data['msno'], columns=['city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'], copy = True).set_index(train_data.index)
song_reorder_train = song_data.set_index('song_id').reindex(train_data['song_id'], columns=['song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist', 'language'], copy = True).set_index(train_data.index)
song_extra_reorder_train = song_extra_data.set_index('song_id').reindex(train_data['song_id'], columns=['name', 'isrc'], copy = True).set_index(train_data.index)

In [12]:
member_reorder_test = member_data.set_index('msno').reindex(test_data['msno'], columns=['city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'], copy = True).set_index(test_data.index)
song_reorder_test = song_data.set_index('song_id').reindex(test_data['song_id'], columns=['song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist', 'language'], copy = True).set_index(test_data.index)
song_extra_reorder_test = song_extra_data.set_index('song_id').reindex(test_data['song_id'], columns=['name', 'isrc'], copy = True).set_index(test_data.index)

In [13]:
all_train_data = pd.concat([train_data, member_reorder_train, song_reorder_train, song_extra_reorder_train], join='inner', axis=1, copy=True)
all_test_data = pd.concat([test_data, member_reorder_test, song_reorder_test, song_extra_reorder_test], join='inner', axis=1, copy=True)

In [14]:
all_train_data.to_csv(DATASET_PATH + 'all_train_withextra.csv', index=False)
all_test_data.to_csv(DATASET_PATH + 'all_test_withextra.csv', index=False)

In [15]:
composer_df = split_and_strip_append(song_data, 'composer', 5, DATASET_PATH + 'all_composer.csv')
print('all_composer saved.')

all_composer saved.


In [16]:
artist_name_df = split_and_strip_append(song_data, 'artist_name', 4, DATASET_PATH + 'all_artist_name.csv')
print('all_artist_name saved.')

all_artist_name saved.


In [17]:
lyricist_df = split_and_strip_append(song_data, 'lyricist', 6, DATASET_PATH + 'all_lyricist.csv')
print('all_lyricist saved.')

all_lyricist saved.


In [18]:
print(train_data.index)

RangeIndex(start=0, stop=7377418, step=1)


In [19]:
print(member_data.head())

                                           msno  city  bd gender  \
0  XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=     1   0    NaN   
1  UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=     1   0    NaN   
2  D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=     1   0    NaN   
3  mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=     1   0    NaN   
4  q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=     1   0    NaN   

   registered_via  registration_init_time  expiration_date  
0               7                20110820         20170920  
1               7                20150628         20170622  
2               4                20160411         20170712  
3               9                20150906         20150907  
4               4                20170126         20170613  


In [20]:
print(all_train_data.head())

                                           msno  \
0  FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   
1  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
2  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
3  Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
4  FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   

                                        song_id source_system_tab  \
0  BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=           explore   
1  bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=        my library   
2  JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=        my library   
3  2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=        my library   
4  3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=           explore   

    source_screen_name      source_type  target  city  bd  gender  \
0              Explore  online-playlist       1     1   0     NaN   
1  Local playlist more   local-playlist       1    13  24  female   
2  Local playlist more   local-playlist       1    13  24  female   
3 

In [21]:
print(all_test_data.head())

   id                                          msno  \
0   0  V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=   
1   1  V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=   
2   2  /uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=   
3   3  1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=   
4   4  1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=   

                                        song_id source_system_tab  \
0  WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=        my library   
1  y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=        my library   
2  8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=          discover   
3  ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=             radio   
4  MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=             radio   

    source_screen_name          source_type  city  bd gender  registered_via  \
0  Local playlist more        local-library     1   0    NaN               7   
1  Local playlist more        local-library     1   0    NaN               7   
2             

In [22]:
print(all_train_data)

                                                 msno  \
0        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   
1        Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
2        Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
3        Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
4        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   
5        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   
6        Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=   
7        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=   
8        uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE=   
9        uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE=   
10       uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE=   
11       uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE=   
12       TJU0Gfvy7FB+r89bWovPKXTjuApTCiv3xg/tt5shR78=   
13       3g0bC24RD7QUeALY1IF/e8L6X3fHG298ZFIsCEG9T9Y=   
14       TJU0Gfvy7FB+r89bWovPKXTjuApTCiv3xg/tt5shR78=   
15       3g0bC24RD7QUeALY1IF/e8L6X3fHG298ZFIsCEG9T9Y=   
16       TJU0Gfvy7FB+r89bWovPKX